In [44]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine


## Extracting Data From Sources 

In [3]:
df1=pd.read_csv("Crop_recommendation.csv")
df2=pd.read_csv("Agrofood_co2_emission.csv")
df1.head()


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [4]:
df2.head()

,Area,Year,Savanna fires,Forest fires,Crop Residues,Rice Cultivation,Drained organic soils (CO2),Pesticides Manufacturing,Food Transport,Forestland,...,Manure Management,Fires in organic soils,Fires in humid tropical forests,On-farm energy use,Rural population,Urban population,Total Population - Male,Total Population - Female,total_emission,Average Temperature °C
0,Afghanistan,1990,14.7237,0.0557,205.6077,686.00,0.0,11.807483,63.1152,-2388.803,...,319.1763,0.0,0.0,NaN,9655167.0,2593947.0,5348387.0,5346409.0,2198.963539,0.536167
1,Afghanistan,1991,14.7237,0.0557,209.4971,678.16,0.0,11.712073,61.2125,-2388.803,...,342.3079,0.0,0.0,NaN,10230490.0,2763167.0,5372959.0,5372208.0,2323.876629,0.020667
2,Afghanistan,1992,14.7237,0.0557,196.5341,686.00,0.0,11.712073,53.3170,-2388.803,...,349.1224,0.0,0.0,NaN,10995568.0,2985663.0,6028494.0,6028939.0,2356.304229,-0.259583
3,Afghanistan,1993,14.7237,0.0557,230.8175,686.00,0.0,11.712073,54.3617,-2388.803,...,352.2947,0.0,0.0,NaN,11858090.0,3237009.0,7003641.0,7000119.0,2368.470529,0.101917
4,Afghanistan,1994,14.7237,0.0557,242.0494,705.60,0.0,11.712073,53.9874,-2388.803,...,367.6784,0.0,0.0,NaN,12690115.0,3482604.0,7733458.0,7722096.0,2500.768729,0.372250


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965 entries, 0 to 6964
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Area                             6965 non-null   object 
 1   Year                             6965 non-null   int64  
 2   Savanna fires                    6934 non-null   float64
 3   Forest fires                     6872 non-null   float64
 4   Crop Residues                    5576 non-null   float64
 5   Rice Cultivation                 6965 non-null   float64
 6   Drained organic soils (CO2)      6965 non-null   float64
 7   Pesticides Manufacturing         6965 non-null   float64
 8   Food Transport                   6965 non-null   float64
 9   Forestland                       6472 non-null   float64
 10  Net Forest conversion            6472 non-null   float64
 11  Food Household Consumption       6492 non-null   float64
 12  Food Retail         

In [6]:
df1.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB


In [8]:
df2.nunique()

Area                                236
Year                                 31
Savanna fires                      3746
Forest fires                       2962
Crop Residues                      5358
Rice Cultivation                   3635
Drained organic soils (CO2)        2147
Pesticides Manufacturing           1310
Food Transport                     6512
Forestland                          605
Net Forest conversion               442
Food Household Consumption         6358
Food Retail                        6459
On-farm Electricity Use            5322
Food Packaging                     3704
Agrifood Systems Waste Disposal    6419
Food Processing                    3542
Fertilizers Manufacturing          2973
IPPU                               6026
Manure applied to Soils            5927
Manure left on Pasture             5922
Manure Management                  5931
Fires in organic soils              126
Fires in humid tropical forests    1736
On-farm energy use                 5558


In [9]:
df2['population'] = df2['Total Population - Male'] + df2['Total Population - Female']
df2.drop(columns=['Total Population - Male', 'Total Population - Female'], inplace=True)
df2['emission_per_person'] = df2.total_emission / df2.population


In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965 entries, 0 to 6964
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Area                             6965 non-null   object 
 1   Year                             6965 non-null   int64  
 2   Savanna fires                    6934 non-null   float64
 3   Forest fires                     6872 non-null   float64
 4   Crop Residues                    5576 non-null   float64
 5   Rice Cultivation                 6965 non-null   float64
 6   Drained organic soils (CO2)      6965 non-null   float64
 7   Pesticides Manufacturing         6965 non-null   float64
 8   Food Transport                   6965 non-null   float64
 9   Forestland                       6472 non-null   float64
 10  Net Forest conversion            6472 non-null   float64
 11  Food Household Consumption       6492 non-null   float64
 12  Food Retail         

In [11]:
relevant_columns = [
    'Area', 'Year', 'Rural population', 'Urban population', 'population', 'total_emission','emission_per_person' ,'Average Temperature °C',
     'Crop Residues', 'Rice Cultivation','Manure Management','Manure left on Pasture','Manure applied to Soils'
]

## Fetching only these columns in df2

df2=df2[relevant_columns]

In [12]:
df2.head()

,Area,Year,Rural population,Urban population,population,total_emission,emission_per_person,Average Temperature °C,Crop Residues,Rice Cultivation,Manure Management,Manure left on Pasture,Manure applied to Soils
0,Afghanistan,1990,9655167.0,2593947.0,10694796.0,2198.963539,0.000206,0.536167,205.6077,686.00,319.1763,1590.5319,260.1431
1,Afghanistan,1991,10230490.0,2763167.0,10745167.0,2323.876629,0.000216,0.020667,209.4971,678.16,342.3079,1657.2364,268.6292
2,Afghanistan,1992,10995568.0,2985663.0,12057433.0,2356.304229,0.000195,-0.259583,196.5341,686.00,349.1224,1653.5068,264.7898
3,Afghanistan,1993,11858090.0,3237009.0,14003760.0,2368.470529,0.000169,0.101917,230.8175,686.00,352.2947,1642.9623,261.7221
4,Afghanistan,1994,12690115.0,3482604.0,15455554.0,2500.768729,0.000162,0.372250,242.0494,705.60,367.6784,1689.3593,267.6219


In [13]:
def check_unique_columns(df):
    unique_counts = df.nunique()
    total_rows = df.shape[0]
    unique_columns = unique_counts[unique_counts == total_rows].index.tolist()
    return unique_columns

unique_columns_df1 = check_unique_columns(df1)
print("Unique columns in df1:", unique_columns_df1)

unique_columns_df2 = check_unique_columns(df2)
print("Unique columns in df2:", unique_columns_df2)


Unique columns in df1: ['temperature', 'humidity', 'ph', 'rainfall']
Unique columns in df2: []


In [14]:
df2.isnull().sum()

Area                          0
Year                          0
Rural population              0
Urban population              0
population                    0
total_emission                0
emission_per_person           0
Average Temperature °C        0
Crop Residues              1389
Rice Cultivation              0
Manure Management           928
Manure left on Pasture        0
Manure applied to Soils     928
dtype: int64

In [15]:
df1.isnull().sum()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

In [16]:
df2.Area.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belgium-Luxembourg', 'Belize', 'Benin', 'Bermuda',
       'Bhutan', 'Bolivia (Plurinational State of)',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Channel Islands', 'Chile', 'China', 'China, Hong Kong SAR',
       'China, Macao SAR', 'China, mainland', 'China, Taiwan Province of',
       'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica',
       'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Czechoslovakia',
       "Democratic People's Republic of Korea",
       'Democratic 

In [17]:
df2['Area'].replace({'United States of America': 'United States'}, inplace=True)
df2['Area'].replace({'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom'}, inplace=True)


In [53]:
def data_loader(data_frame, table_name):
    engine = create_engine("sqlite:///c:/Users/farrukh/Documents/made-template/data/Carbon_emission.sqlite")
    data_frame.to_sql(table_name,engine, if_exists="replace", index=False)
    

In [54]:
data_loader(df2,"agro_food")
data_loader(df1,"smart_agriculture")